<a href="https://colab.research.google.com/github/pragneshrana/WorkSample/blob/main/WorkSample/LLM/transformer_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tokenization
- Word tokenization
- Char tokenization
- Sub-word tokenization

In [ ]:
!pip install seqeval
!pip install wandb
!pip install transformers==4.30


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=8244ea2ebb84d462c59e03d3cf03c323ee25504286e59e9a28c0aadbbd20f9a5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install datasets tokenizers

['Collecting datasets',
 '  Downloading datasets-2.14.6-py3-none-any.whl (493 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/493.7 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b81.9/493.7 kB\x1b \x1b2.5 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b491.5/493.7 kB\x1b \x1b8.0 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b493.7/493.7 kB\x1b \x1b6.9 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hCollecting tokenizers',
 '  Downloading tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/3.8 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━\x1b \x1b2.5/3.8 MB\x1b \x1b66.7 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikiann", "gu")


In [ ]:
dataset['train'][0]

{'tokens': ['વીરપુર', '(', 'રાજકોટ', ')'],
 'ner_tags': [5, 6, 6, 6],
 'langs': ['gu', 'gu', 'gu', 'gu'],
 'spans': ['LOC: વીરપુર ( રાજકોટ )']}

tokens - tokenized words

ner_tags - Numerical Tagges assigned to each word

langs -  Language of each word

spans - Dictionary of tag and word

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names


For input we need to provide input_ids, token_type_ids and attention_mask as well as we need to adjust the labels.

In [ ]:
from transformers import AutoTokenizer
#tokenizer is responsible for encoding text into tokens that the BERT model can understand.
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")



In [ ]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True) #tokenize the batch sample

  total_adjusted_labels = [] #store adjusted labels for each tokenized sample.
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    #looping over each tokenized dataset sample
    prev_wid = -1
    #wordID and labelID for given sample
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]

    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100


In [ ]:
#Different sample get tokenized to different length so we need to provide
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import os
import wandb
os.environ["WANDB_API_KEY"]="44f1ba17c0a33c2f9b9155a819ff7e8737d25e96"
os.environ["WANDB_ENTITY"]="pragnesh-aitest"
os.environ["WANDB_PROJECT"]="finetune_bert_ner"

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results


<ipython-input-8-c3947d92ad66>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_names))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and a

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    run_name = "ep_10_tokenized_11",
    save_strategy='no'
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: pragnesh-aitest. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=49, training_loss=0.7747364433444276, metrics={'train_runtime': 718.2905, 'train_samples_per_second': 0.975, 'train_steps_per_second': 0.068, 'total_flos': 29515619409936.0, 'train_loss': 0.7747364433444276, 'epoch': 7.0})

In [ ]:
wandb.finish()

train/epoch,▁
train/global_step,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,7.0
train/global_step,49
train/total_flos,29515619409936.0
train/train_loss,0.77474
